In [1]:
import time

import osu.rulesets.beatmap as bm
import osu.rulesets.replay as rp
import osu.dataset as dataset
import numpy as np
import torch

test_name = 'rubiks'

test_map = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'

test_map = bm.load(test_map)

In [2]:
# dataset.replay_mapping_from_cache()

In [3]:
# data = dataset.input_data(test_map)
# data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
# data = torch.FloatTensor(data).to('cuda')

In [ ]:
replay_data = np.load(f".generated/{test_name}.npy")
replay_data

In [5]:
import osu.preview.preview as preview
print(test_map)
test_map.apply_mods(['hr'])
preview.preview_replay(replay_data, test_map, test_song)

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
# alt_data = dataset.replay_to_output_data(test_map, test_replay)
# alt_data = np.concatenate(alt_data)
# 
# alt_data

In [6]:
# actually play out the replay
import win32gui
import mouse
import time
import osu.rulesets.core as osu_core
import pyautogui

def active_win_name():
    hwnd = win32gui.GetForegroundWindow()
    return win32gui.GetWindowText(hwnd)

def osu_to_screen_pixel(x, y) -> (int, int):
    w, h = pyautogui.size()
    
    # screen/osu ratio
    # https://osu.ppy.sh/wiki/en/Client/Playfield
    osr_x = w / 640 * 0.75 # for some reason this constant just works. misleading wiki?
    osr_y = h / 480
    
    x = x * osr_x
    y = y * osr_y
    
    # osu width and height
    ow = osu_core.SCREEN_WIDTH * osr_x
    oh = osu_core.SCREEN_HEIGHT * osr_y
    
    offset_x = 0.5 * (w - ow)
    # "The playfield is slightly shifted vertically, placed 8 game pixels lower than the window's centre."
    offset_y = 0.5 * (h - oh) + (8 * osr_y)
    
    return int(x + offset_x), int(y + offset_y)

time.sleep(1.25)
win_name = active_win_name()

target_interval = 0.024  # 24ms per frame
start_time = time.perf_counter()
frame_count = 0

for x, y in replay_data:
    if win_name != active_win_name():
        break
    x = (x + 0.5) * osu_core.SCREEN_WIDTH
    y = (y + 0.5) * osu_core.SCREEN_HEIGHT
    (x, y) = osu_to_screen_pixel(x, y)
    mouse.move(x, y)
    
    frame_count += 1
    
    target_time = start_time + (frame_count * target_interval)
    current_time = time.perf_counter()
    
    # run at constant rate 
    sleep_time = target_time - current_time
    if sleep_time > 0:
        time.sleep(sleep_time)
